In [1]:
import pandas as pd
from datetime import datetime

pd.options.display.max_rows=500
pd.options.display.max_rows=500

In [2]:
import os

In [3]:
from pycountry import countries

In [4]:
pwd

'/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/jupyter_notebooks'

In [5]:
cd ../python_scripts/

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts


In [6]:
from hopkins_cleaner import HopkinsDataFull

In [7]:
self = HopkinsDataFull()

In [8]:
self.run()

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/us_data_cleaner.py:113: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.data = pd.concat([df1, df2], axis=0)
/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/hopkins_cleaner.py:252: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ].rank(ascending=True)
/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/hopkins_cleaner.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [9]:
df = self.data

In [10]:
df.loc[df['province_or_state'] == 'VA'].tail(10)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,hundred_case_state_rank,hundred_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg
13454,US,2020-03-06,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,-78.024902,NaN,33.0,NaN,NaN,US,USA,327167434.0,38.1,262
13455,US,2020-03-07,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,-78.024902,NaN,34.0,NaN,1.0,US,USA,327167434.0,38.1,402
13456,US,2020-03-08,VA,2,VA-US,0.0,2.0,0,0.0,0.0,...,-78.024902,NaN,35.0,NaN,2.0,US,USA,327167434.0,38.1,518
13457,US,2020-03-09,VA,2,VA-US,2.0,0.0,0,0.0,0.0,...,-78.024902,NaN,36.0,NaN,3.0,US,USA,327167434.0,38.1,583
14004,US,2020-03-10,VA,7,VA-US,2.0,5.0,0,0.0,0.0,...,-78.024902,1.0,37.0,NaN,4.0,US,USA,327167434.0,38.1,959
14005,US,2020-03-11,VA,9,VA-US,7.0,2.0,0,0.0,0.0,...,-78.024902,2.0,38.0,NaN,5.0,US,USA,327167434.0,38.1,1281
14006,US,2020-03-12,VA,17,VA-US,9.0,8.0,0,0.0,0.0,...,-78.024902,3.0,39.0,NaN,6.0,US,USA,327167434.0,38.1,1663
14007,US,2020-03-13,VA,30,VA-US,17.0,13.0,0,0.0,0.0,...,-78.024902,4.0,40.0,NaN,7.0,US,USA,327167434.0,38.1,2179
14008,US,2020-03-14,VA,41,VA-US,30.0,11.0,0,0.0,0.0,...,-78.024902,5.0,41.0,NaN,8.0,US,USA,327167434.0,38.1,2727
14009,US,2020-03-15,VA,45,VA-US,41.0,4.0,0,0.0,0.0,...,-78.024902,6.0,42.0,NaN,9.0,US,USA,327167434.0,38.1,3499


In [ ]:
df.loc[df['province_or_state'].str.contains('DC')]

In [ ]:
yesterday.sort_values('running_total_cases', ascending=False)[['country_or_region','running_total_cases','running_total_recoveries','running_total_deaths','country_population_2018','country_median_age']]

In [21]:
df.to_csv('../output_data/HOPKINS_CLEANED.csv', index=False)

# QA 

#### Make sure all US states have the same number of days 

In [11]:
us = df.loc[df['country_or_region'] == 'US']

In [12]:
us.groupby(['province_or_state'])['daily_new_cases'].size().reset_index()

,province_or_state,daily_new_cases
0,AK,54
1,AL,54
2,AR,54
3,AZ,54
4,CA,54
5,CO,54
6,CT,54
7,DC,54
8,DE,54
9,Diamond Princess,54


#### Total US cases 

In [13]:
us['daily_new_cases'].sum()

3499.0

In [14]:
df.head(2)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,hundred_case_state_rank,hundred_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg
0,Thailand,2020-01-22,Not Provided,2,Not Provided-Thailand,0.0,2.0,0,0.0,0.0,...,101.0,NaN,NaN,NaN,NaN,TH,THA,69428524.0,37.7,2
1,Thailand,2020-01-23,Not Provided,3,Not Provided-Thailand,2.0,1.0,0,0.0,0.0,...,101.0,NaN,NaN,NaN,NaN,TH,THA,69428524.0,37.7,3


In [15]:
max_date = df['date'].max()

In [16]:
yesterday = df.loc[df['date'] == max_date]
yesterday['running_total_cases'].sum(), yesterday['running_total_recoveries'].sum(), yesterday['running_total_deaths'].sum()

(167449, 76034, 6440)

In [17]:
df['daily_new_cases'].sum(), df['daily_new_recoveries'].sum(), df['daily_new_deaths'].sum()

(167449.0, 76034.0, 6440.0)

In [18]:
df.loc[df['province_or_state'] == 'WA'].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,hundred_case_state_rank,hundred_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg
14017,US,2020-03-11,WA,366,WA-US,267.0,99.0,1,1.0,0.0,...,-120.740135,12.0,38.0,5.0,5.0,US,USA,327167434.0,38.1,1281
14018,US,2020-03-12,WA,442,WA-US,366.0,76.0,1,1.0,0.0,...,-120.740135,13.0,39.0,6.0,6.0,US,USA,327167434.0,38.1,1663
14019,US,2020-03-13,WA,568,WA-US,442.0,126.0,1,1.0,0.0,...,-120.740135,14.0,40.0,7.0,7.0,US,USA,327167434.0,38.1,2179
14020,US,2020-03-14,WA,572,WA-US,568.0,4.0,1,1.0,0.0,...,-120.740135,15.0,41.0,8.0,8.0,US,USA,327167434.0,38.1,2727
14021,US,2020-03-15,WA,643,WA-US,572.0,71.0,1,1.0,0.0,...,-120.740135,16.0,42.0,9.0,9.0,US,USA,327167434.0,38.1,3499


In [19]:
df.loc[df['province_or_state'] == 'VA'].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,hundred_case_state_rank,hundred_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg
14005,US,2020-03-11,VA,9,VA-US,7.0,2.0,0,0.0,0.0,...,-78.024902,2.0,38.0,NaN,5.0,US,USA,327167434.0,38.1,1281
14006,US,2020-03-12,VA,17,VA-US,9.0,8.0,0,0.0,0.0,...,-78.024902,3.0,39.0,NaN,6.0,US,USA,327167434.0,38.1,1663
14007,US,2020-03-13,VA,30,VA-US,17.0,13.0,0,0.0,0.0,...,-78.024902,4.0,40.0,NaN,7.0,US,USA,327167434.0,38.1,2179
14008,US,2020-03-14,VA,41,VA-US,30.0,11.0,0,0.0,0.0,...,-78.024902,5.0,41.0,NaN,8.0,US,USA,327167434.0,38.1,2727
14009,US,2020-03-15,VA,45,VA-US,41.0,4.0,0,0.0,0.0,...,-78.024902,6.0,42.0,NaN,9.0,US,USA,327167434.0,38.1,3499


In [20]:
df.loc[df['province_or_state'] == 'CA'].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,hundred_case_state_rank,hundred_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg
13723,US,2020-03-11,CA,177,CA-US,144.0,33.0,2,2.0,0.0,...,-119.417931,38.0,38.0,3.0,5.0,US,USA,327167434.0,38.1,1281
13724,US,2020-03-12,CA,221,CA-US,177.0,44.0,6,2.0,4.0,...,-119.417931,39.0,39.0,4.0,6.0,US,USA,327167434.0,38.1,1663
13725,US,2020-03-13,CA,282,CA-US,221.0,61.0,6,6.0,0.0,...,-119.417931,40.0,40.0,5.0,7.0,US,USA,327167434.0,38.1,2179
13726,US,2020-03-14,CA,340,CA-US,282.0,58.0,6,6.0,0.0,...,-119.417931,41.0,41.0,6.0,8.0,US,USA,327167434.0,38.1,2727
13727,US,2020-03-15,CA,426,CA-US,340.0,86.0,6,6.0,0.0,...,-119.417931,42.0,42.0,7.0,9.0,US,USA,327167434.0,38.1,3499


## QA of Data 

In [ ]:
# This indicates a shift in the data
df.loc[df['daily_diff_cases'] < 0]['date'].value_counts()

In [ ]:
# This indicates a shift in the data
df_final.loc[df_final['daily_diff_cases'] < 0]['province_state'].value_counts()

In [ ]:
df_final.loc[df_final['daily_diff_cases'] < 0]

In [ ]:
df_final.loc[(df_final['daily_diff_cases'] < 0) &
      (df_final['province_state'] == 'CA')]

In [ ]:
df_final.loc[df_final['country_region'] == 'China'].head(200).sort_values('rank')

In [ ]:
df_final.country_region.value_counts()